In [1]:
from buildingmotif.shape_builder.shape import Shape, NodeShape, PropertyShape, OR, AND, NOT, XONE
from buildingmotif.namespaces import BRICK, SH, bind_prefixes
from rdflib.namespace import Namespace
from rdflib import Graph, URIRef
BLDG = Namespace("urn:building")
SHAPES = Namespace("urn:shapes#")

In [2]:
shapes = Graph()
zone_with_room = (NodeShape(SHAPES["zone-with-room"])
    .of_class(BRICK["HVAC_Zone"])
    .has_property(PropertyShape()
        .has_path(BRICK["hasPart"])
        .matches_shape(NodeShape()
            .of_class(BRICK["Room"])
            .has_property(PropertyShape()
                .has_path(BRICK["hasPoint"])
                .matches_shape(NodeShape()
                    .of_class(BRICK["Open_Close_Status"]),
                    min=1
                )
            ),
            min=1
        )
    ))
shapes += zone_with_room
window_switch = (NodeShape(SHAPES["window-switch"])
    .has_property(PropertyShape()
        .has_path(BRICK["feeds"], one_or_more=True)
        .matches_shape(zone_with_room, min=0)
        )
    )
shapes += window_switch
zone_with_occ_sensor = (NodeShape(SHAPES["zone-with-occ-sensor"])
    .of_class(BRICK["HVAC_Zone"])
    .has_property(
        PropertyShape()
        .has_path(BRICK["hasPoint"])
        .matches_class(BRICK["Occupancy_Sensor"], min=1)
    )
)
shapes += zone_with_occ_sensor
room_with_occ_sensor = (NodeShape(SHAPES["room-with-occ-sensor"])
    .of_class(BRICK["Room"])
    .has_property(
        PropertyShape()
        .has_path(BRICK["hasPoint"])
        .matches_class(BRICK["Occupancy_Sensor"], min=1)
    )
)
shapes += room_with_occ_sensor
occupancy_sensor_1 = (PropertyShape(SHAPES["ocupancy-sensor-1"])
    .has_path(BRICK["hasPoint"])
    .matches_class(BRICK["Occcupancy_Sensor"], min=0)
    )
shapes += occupancy_sensor_1
occupancy_sensor_2 = (PropertyShape(SHAPES["occupancy-sensor-2"])
    .has_path(BRICK["feeds"], one_or_more=True)
    .has_path(BRICK["hasPart"], zero_or_one=True)
    .matches_shape(OR(zone_with_occ_sensor, room_with_occ_sensor), min=0)
)
shapes += occupancy_sensor_2
occupancy_sensor = NodeShape(SHAPES["occupancy-sensor"]).OR(occupancy_sensor_1, occupancy_sensor_2)
shapes += occupancy_sensor
print((shapes.serialize(format="turtle")))


@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<urn:shapes#occupancy-sensor> a sh:NodeShape ;
    sh:or ( <urn:shapes#ocupancy-sensor-1> <urn:shapes#occupancy-sensor-2> ) .

<urn:shapes#window-switch> a sh:NodeShape ;
    sh:property [ a sh:PropertyShape ;
            sh:minCount 0 ;
            sh:node <urn:shapes#zone-with-room> ;
            sh:path [ sh:path-one-or-more brick:feeds ] ] .

<urn:shapes#occupancy-sensor-2> a sh:PropertyShape ;
    sh:minCount 0 ;
    sh:node [ sh:or ( <urn:shapes#zone-with-occ-sensor> <urn:shapes#room-with-occ-sensor> ) ] ;
    sh:path [ sh:path-one-or-more brick:feeds ],
        [ sh:path-zero-or-one brick:hasPart ] .

<urn:shapes#ocupancy-sensor-1> a sh:PropertyShape ;
    sh:class brick:Occcupancy_Sensor ;
    sh:minCount 0 ;
    sh:path brick:hasPoint .

<urn:shapes#room-with-occ-sen

In [3]:
print(NodeShape(URIRef("http://example.org/shapes#shape"), "hello").serialize())

@prefix sh: <http://www.w3.org/ns/shacl#> .

<http://example.org/shapes#shape> a sh:NodeShape ;
    sh:message "hello" .


